In [4]:
import pandas as pd
import numpy as np

movie_columns = ['movie_id', 'title']
rating_columns = ['user_id', 'movie_id', 'rating']


movies = pd.read_csv( 'ml-100k/u.item', sep='|', names = movie_columns, usecols = range(2), encoding='ISO-8859-1'  )
ratings = pd.read_csv( 'ml-100k/u.data', sep='\t', names = rating_columns, usecols = range(3), encoding='ISO-8859-1'  )

ratings = pd.merge( movies, ratings )

In [5]:
# movie ratings pivot against user id and movie for movie rating.

pivot_movie_rating = ratings.pivot_table( index = 'user_id', columns = 'title', values = 'rating' )

In [6]:
StarWarsRatings = pivot_movie_rating['Star Wars (1977)']

Using corrwith() function to compute the pairwise correlation of user ratings of star wars vector with every other movie.
Then wl drop the results which doesnt have at data and construct a DF of movies and their correlation(simmilarity) scores with star wars.

In [7]:
simmilarMovies = pivot_movie_rating.corrwith( StarWarsRatings )
simmilarMovies = simmilarMovies.dropna()
dataFrame = pd.DataFrame( simmilarMovies )
dataFrame.head()
# Data Frame above prints the results of movies which are not simmilar to Star Wars, this means we are missing something!!

C:\Users\swapnil.shinde\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\numpy\lib\function_base.py:3159: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\Users\swapnil.shinde\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\numpy\lib\function_base.py:3093: RuntimeWarning: divide by zero encountered in double_scalars
  c *= 1. / np.float64(fact)


,0
title,
'Til There Was You (1997),0.872872
1-900 (1994),-0.645497
101 Dalmatians (1996),0.211132
12 Angry Men (1957),0.184289
187 (1997),0.027398


In [8]:
simmilarMovies.sort_values( ascending = False )

title
No Escape (1994)                                                                     1.000000
Man of the Year (1995)                                                               1.000000
Hollow Reed (1996)                                                                   1.000000
Commandments (1997)                                                                  1.000000
Cosi (1996)                                                                          1.000000
Stripes (1981)                                                                       1.000000
Golden Earrings (1947)                                                               1.000000
Mondo (1996)                                                                         1.000000
Line King: Al Hirschfeld, The (1996)                                                 1.000000
Outlaw, The (1943)                                                                   1.000000
Hurricane Streets (1998)                              

Lets create the DF for that counts how many ratings exists for each movie and the average of it.

In [9]:
movieStats = ratings.groupby( 'title' ).agg({'rating' : [np.size, np.mean]})

# drop all movies with less than 150 ratings
popularMovies = movieStats['rating']['size'] > 150
movieStats[popularMovies].sort_values([('rating','mean')],ascending=False)[:15]

rating          
                                                     size      mean
title                                                              
Schindler's List (1993)                               298  4.466443
Casablanca (1942)                                     243  4.456790
Shawshank Redemption, The (1994)                      283  4.445230
Rear Window (1954)                                    209  4.387560
Usual Suspects, The (1995)                            267  4.385768
Star Wars (1977)                                      584  4.359589
Citizen Kane (1941)                                   198  4.292929
To Kill a Mockingbird (1962)                          219  4.292237
One Flew Over the Cuckoo's Nest (1975)                264  4.291667
Silence of the Lambs, The (1991)                      390  4.289744
North by Northwest (1959)                             179  4.284916
Godfather, The (1972)                                 413  4.283293
Secrets & Lies (1996)                                 162  4.265432
Good Will Hunting (1997)                              198  4.262626
Dr. Strangelove or: How I Learned to Stop Worry...    194  4.252577

In [10]:
df = movieStats[popularMovies].join( pd.DataFrame( simmilarMovies, columns =['similarity'] ) )
df.head()

C:\Users\swapnil.shinde\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\pandas\core\reshape\merge.py:551: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


,"(rating, size)","(rating, mean)",similarity
title,,,
2001: A Space Odyssey (1968),259,3.969112,0.230884
"Abyss, The (1989)",151,3.589404,0.203709
"African Queen, The (1951)",152,4.184211,0.230540
Air Force One (1997),431,3.631090,0.113164
Aladdin (1992),219,3.812785,0.191621


In [11]:
#sort by similarity score
df.sort_values(['similarity'], ascending = False )[:15]

,"(rating, size)","(rating, mean)",similarity
title,,,
Star Wars (1977),584,4.359589,1.000000
"Empire Strikes Back, The (1980)",368,4.206522,0.748353
Return of the Jedi (1983),507,4.007890,0.672556
Raiders of the Lost Ark (1981),420,4.252381,0.536117
"Sting, The (1973)",241,4.058091,0.367538
Indiana Jones and the Last Crusade (1989),331,3.930514,0.350107
L.A. Confidential (1997),297,4.161616,0.319065
"Bridge on the River Kwai, The (1957)",165,4.175758,0.316580
E.T. the Extra-Terrestrial (1982),300,3.833333,0.303619
